In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("../input/massive-stock-news-analysis-db-for-nlpbacktests/analyst_ratings_processed.csv",error_bad_lines=False)
data.sample(5).head(5)
data = data.drop_duplicates()

In [ ]:
def getUniqueness(dataset):
    for (columnName, columnData) in dataset.iteritems():
        print(f"unique values in [{columnName}]: {columnData.nunique()}")

    print(f"total rows: {len(dataset.index)}")
getUniqueness(data)

In [ ]:
stockseries =data["stock"].value_counts()[:400].index.tolist()
data = data[data["stock"].isin(stockseries)]
getUniqueness(data)

In [ ]:
import requests
def sendRequest(stock):
    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-chart"

    querystring = {"interval":"1d","symbol":stock,"range":"10y","region":"US"}

    headers = {
        'x-rapidapi-key': "45d667af06msheb5f2732d2eef1bp1352f5jsn5b6c64d88be2",
        'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    if response.status_code==200:
        return response.text
    else:
        return None

In [ ]:
import json
def getHistData(stock):
    response = sendRequest(stock)
    histdata = pd.DataFrame()
    if response == None:
        return histdata
    else:
        try:
            jsondata = json.loads(response)

            #print(data2['chart']["result"][0]["indicators"]["quote"][0])
            timestamps = pd.Series(jsondata['chart']["result"][0]["timestamp"])
            high = pd.Series(jsondata['chart']["result"][0]["indicators"]["quote"][0]["high"])
            low = pd.Series(jsondata['chart']["result"][0]["indicators"]["quote"][0]["low"])
            close = pd.Series(jsondata['chart']["result"][0]["indicators"]["adjclose"][0]["adjclose"])
            columns = {"timestamps":timestamps,"high":high,"low":low,"close":close,"stock":stock}
            histdata = pd.DataFrame(columns)
            return histdata
        except:
            print(stock)
            print(response)
            return pd.DataFrame()
            


In [ ]:
stocks = ["NFLX","BBRY","EBAY"]
frames = []
count = 0
for stock in stockseries:
    histData = getHistData(stock)
    if not histData.empty:
        frames.append(histData)
allstocks = pd.concat(frames)
print(len(allstocks))
allstocks.sample(20).head(20)


In [ ]:
allstocks.to_csv("stockhistory.csv",index = False)